---
* *** 원본 소스는 홍콩과기대 김성훈 교수님의  [Github link](https://github.com/hunkim/DeepLearningZeroToAll)를 참조하세요 ***     
* *** Watson studio notebook 에서 작업 가능하도록 수정하였으며***
* *** 한글로 설명된 부분은 제가 스터디 하면서 이해한 내용을 추가한 것입니다. ***
---

# Lab 9-4 XOR Tensorboard

### - tensorboard를 띄우기가 어렵네요 (고민중^^ Watson studio에서 불가능할 수도 있음)
* ***os file access해서 web browser를 구동해야함***

In [1]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [2]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [3]:
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [19]:
# Launch graph
sess = tf.Session()
# tensorboard --logdir=./logs/xor_logs
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs")
writer.add_graph(sess.graph)  # Show the graph

# Initialize TensorFlow variables
sess.run(tf.global_variables_initializer())

In [20]:
for step in range(10001):
    summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=step)

    if step % 1000 == 0:
        print(step, sess.run(cost, feed_dict={ X: x_data, Y: y_data}))
print('w1---',sess.run(W1))
print('w2---',sess.run(W2))

0 0.715638
1000 0.022334
2000 0.00630939
3000 0.00277098
4000 0.00142472
5000 0.000789405
6000 0.000455096
7000 0.000268332
8000 0.00016026
9000 9.64897e-05
10000 5.83398e-05
w1--- [[ 9.02439308 -9.65184593]
 [-9.48265171  9.95403957]]
w2--- [[ 20.17106056]
 [ 19.82086754]]


In [6]:
# Accuracy report
h, c, a = sess.run([hypothesis, predicted, accuracy],
                     feed_dict={X: x_data, Y: y_data})
print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


Hypothesis:  [[  6.13103184e-05]
 [  9.99936938e-01]
 [  9.99950767e-01]
 [  5.97514772e-05]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0


In [29]:
!ls -al /home/dsxuser/work/logs

total 10504
drwxr-x--- 3 dsxuser dsxuser     4096 Apr 28 01:52 .
drwxr-x--- 3 dsxuser dsxuser     4096 Apr 28 01:30 ..
-rw-r----- 1 dsxuser dsxuser 10740862 Apr 28 01:52 events.out.tfevents.1524880324.notebook-condafree1x4fecba7770043491cb58d74657965a774-35437h9fv
drwxr-x--- 2 dsxuser dsxuser     4096 Apr 28 01:30 xor_logs_r0_01


In [31]:
!tensorboard --logdir=/home/dsxuser/work/logs

2018-04-28 02:05:57.856119: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-04-28 02:05:57.856193: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-04-28 02:05:57.856215: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-04-28 02:05:57.856243: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-04-28 02:05:57.856262: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

In [ ]:
'''
Hypothesis:  [[ 0.01338218]
 [ 0.98166394]
 [ 0.98809403]
 [ 0.01135799]]
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy:  1.0
'''